<a href="https://colab.research.google.com/github/marianaalvesbr/alura-ai-gemini/blob/main/Assistente_IR_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **DeclaraAI - Seu assistente para declaração do importo de renda**

Fazer uma explicação depoos



1 - Instalando as bibliotecas e instanciando as funcionalidades

In [38]:
#---------------------------------------------------------------#
# Configurações Gerais
#---------------------------------------------------------------#

#Iniciando a instalação da bibliote de GenAI do Gemina
%pip -q install google-genai

# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

# Importar biblioteca para ler PDF
!pip install PyPDF2 # Instala a biblioteca se ainda não tiver

# Configura a API Key do Google Gemini
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# Configura o cliente da SDK do Gemini
from google import genai
client = genai.Client()
MODEL_ID_2 = "gemini-2.0-flash"
MODEL_ID_25 = "gemini-2.5-flash-preview-04-17"

#Importanto funcionalidades que seram uteis para formatação de texto, criação de agent e runner para rodar os agentes
from IPython.display import HTML, Markdown
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

import PyPDF2

# Permitir upload de arquivo no Colab
from google.colab import files


2 - Agora vou criar algumas funções úteis

In [39]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response


# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))


# Função para ler texto de um PDF
def read_pdf(filepath):
    text = ""
    try:
        with open(filepath, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            num_pages = len(reader.pages)
            for page_num in range(num_pages):
                page = reader.pages[page_num]
                text += page.extract_text()
    except Exception as e:
        print(f"Erro ao ler o PDF: {e}")
        text = None # Indica que houve um erro
    return text

# --- Função para limpeza dos outputs dos agentes ---
def clean_agent_output(text):
    # Substitua caracteres problemáticos ou padrões indesejados
    text = text.replace('∗', '') # Exemplo: remover o caractere estrela
    text = text.replace('\n ', '\n') # Exemplo: remover espaços no início de novas linhas (se for um problema)
    # Adicione mais substituições conforme necessário para limpar o texto
    return text

3 - Criando os agentes

In [40]:
###################################################
# --- Agente 1: Buscando informações sobre IR --- #
###################################################

def agente_buscador(ano):
  buscador = Agent(
      name="agente_buscador",
      model=MODEL_ID_2,
      description="Agente que busca informações sobre o imposto de renda do ano incluído pelo usuário",
      tools=[google_search],
      instruction="""
      Você é um agente de pesquisa sobre imposto de renda no Brasil. Sua tarefa é usar a ferramenta de busca do google (google_search)
      para recuperar as últimas informações pertinentes para a declaração do Imposto de Renda com base no ano informado abaixo.

      **Instruções de Formatação da Resposta:**
      - Apresente as informações de forma clara e organizada, usando **Markdown**.
      - Use **títulos e subtítulos** (##, ###) para seções principais (Ex: "Principais Regras", "Prazos", "Novidades").
      - Use **listas com marcadores** (-) para apresentar itens (Ex: Critérios de obrigatoriedade).
      - Use **negrito** (**) para destacar termos importantes (Ex: **Prazo Final**, **Rendimentos Tributáveis**).
      - Evite usar caracteres especiais ou formatação complexa que possa não ser renderizada corretamente.
      - Seja conciso e vá direto ao ponto.
      """
  )

  # Executa o agente
  entrada_do_agente_buscador = f"Ano: {ano}"
  informacoes_buscadas = call_agent(buscador, entrada_do_agente_buscador)
  return informacoes_buscadas





In [41]:
######################################################################
# --- Agente 2: Buscando analisa informações sobre os rendimento --- #
######################################################################

def agente_analisador_rendimentos(ano, pdf_text):
  analisador = Agent(
    name="agente_analisador_rendimentos",
    model=MODEL_ID_25, # Use um modelo mais capaz se possível
    description="Agente que analisa texto de documentos de rendimento e identifica informações relevantes para declaração de imposto de renda.",
    # Pode adicionar ferramentas se precisar pesquisar algo com base nos dados do PDF
    tools=[], # Sem ferramentas por enquanto, focado na análise do texto
    instruction="""
    Você é um assistente especializado em declaração de imposto de renda. Sua tarefa é analisar o texto de documentos de rendimento fornecidos.
    Identifique e liste as seguintes informações, se encontrá-las:
    - Fontes pagadoras (nomes de empresas, bancos, etc.)
    - Tipos de rendimento (salário, aluguéis, aposentadoria, etc.)
    - Valores dos rendimentos
    - Valores de impostos retidos na fonte
    - Outras informações que considere relevantes para a declaração do imposto de renda.
    - Informe o usuário onde no imposto de renda ele deve declarar essas informações

    Liste as informações de rendimento de forma clara, uma informação por linha, sem usar formatação de tabela Markdown.
    """
  )

  # Instanciar o agente analisador
  entrada_do_agente_analisador = f"Ano de refêrencia: {ano}. \n Analise o seguinte texto de um documento de rendimento e extraia as informações relevantes para a declaração de imposto de renda:\n\n{pdf_text}"

  #executa o agente
  analisa_rendimentos = call_agent(analisador, entrada_do_agente_analisador)
  return analisa_rendimentos

In [44]:
#######################################################################
# --- Agente 3: Buscando analisar informações sobre notas fiscais --- #
#######################################################################

def agente_analisador_notas(ano, pdf_text_nota):
  analisador_notas = Agent(
    name="agente_analisador_notas",
    model=MODEL_ID_25, # Use um modelo mais capaz se possível
    description="Agente que analisa texto de documentos de notas fiscais e identifica informações relevantes para declaração de imposto de renda.",

    # Pode adicionar ferramentas se precisar pesquisar algo com base nos dados do PDF
    tools=[], # Sem ferramentas por enquanto, focado na análise do texto

    instruction="""
    Você é um assistente especializado em declaração de imposto de renda. Sua tarefa é analisar o texto de documentos de notas fiscais fornecidos.
    Identifique e liste as seguintes informações, se encontrá-las:
    - Identifique o tipo de nota fiscal, como: serviços de saúde (médico, dentista, psicólogo, etc.), educação (escola, faculdade, cursos),
    ou outros serviços dedutíveis permitidos pela legislação do Imposto de Renda de 2025.
    - Extrair informações do prestador do serviço, como: Nome completo ou razão socual, CPF ou CNPJ, Endereço e outras informações que considerar
    pertinente para facilitar o usuário na declaração
    - Verificar data de emissão e alertar o usuário caso não seja do ano que está declarando.
    - Extrair valores pagos
    - Extrair impostos retidos na fonte
    - Informe a descrição do serviço
    - Outras informações que considere relevantes para a declaração do imposto de renda.
    - Informe o usuário onde no imposto de renda ele deve declarar essas informações

    Apresente um resumo das informações extraídas e diga se ela é provavelmente dedutível, listando os campos que o usuário
    precisará preencher na declaração, sem usar formatação de tabela Markdown.
    """
  )

  # Instanciar o agente analisador
  entrada_do_agente_analisador_notas = f"Ano de refêrencia: {ano}. \n Analise o seguinte texto de um documento de rendimento e extraia as informações relevantes para a declaração de imposto de renda:\n\n{pdf_text_nota}"

  #executa o agente
  analisa_notas = call_agent(analisador_notas, entrada_do_agente_analisador_notas)
  return analisa_notas


In [45]:
#Aplicação Principal

print("🚀 Iniciando o Assistente de Declaração do Imposto de Renda - DeclaraAI 🚀")

# --- Obter o Ano de referência do Usuário ---
ano = input("❓ Por favor, digite o ano que gostaria de consultar a declaração do imposto de renda: ")

# Inserir lógica do sistema de agentes ################################################
if not ano:
  print("Você esqueceu de digitar o ano")
else:
  print(f"Maravilha! Vamos buscar informações principais para sua declaração do imposto de renda de: {ano}. \n\n")
  print("\n-- AGUARDE (Buscando informações gerais sobre IR para o ano) --\n")

  # --- Chamando o primeiro agente que vai trazer as informações básicas sobre o imposto
  informacoes_buscadas = agente_buscador(ano)

  #Fazendo uma limpeza do output
  cleaned_informacoes_buscadas = clean_agent_output(informacoes_buscadas)


  print("\n-- Resultado do Agente com Informações de IR --\n")
  display(to_markdown(cleaned_informacoes_buscadas))
  print("---------------------------------------------\n\n")

  ##########################################################################
  # --- Pedindo ao usuário para fazer upload dos arquivos de rendimento ---#
  ##########################################################################

  print("\n📄 Agora, por favor, faça o upload dos seus arquivos de informe de rendimentos em formato PDF.")
  print("\n👉 Clique no botão 'Escolher arquivos' que aparecerá abaixo para selecionar os documentos.")
  print("\n✅ Você pode enviar um ou mais arquivos de uma vez.")

  #solicitando o arquivo
  upload_rendimentos = files.upload()

  if not upload_rendimentos:
    print("❌ Nenhum arquivo foi enviado. Por favor, tente novamente.")
  else:
    # --- Informar nome dos arquivos ---
    print("Maravilha! Recebemos seus arquivos.")

    # informando o nome dos arquivos enviados
    print("\nEsses foram os arquivos enviados e que vamos avaliar")
    for filename in upload_rendimentos.keys():
        print(f"- {filename}")
    print("-" * 30) # Linha separadora para clareza

    # Mensagem de espera mais específica
    print("\n-- AGUARDE (Analisando rendimentos dos arquivos enviados) --\n")

    # Lista para armazenar os resultados da análise de cada arquivo
    resultados_analise_arquivos = []

    for filename in upload_rendimentos.keys(): # Itera sobre todos os arquivos enviados
        print(f"Processando arquivo: {filename}...") # Informa qual arquivo está processando
        pdf_text = read_pdf(filename) # Lê o texto deste arquivo específico

        if pdf_text:
            # Chama o agente analisador para CADA arquivo
            print(f"  ✅ Texto extraído. Enviando para análise do agente...")
            analise_individual = agente_analisador_rendimentos(ano, pdf_text)

            # Armazenar o resultado
            resultados_analise_arquivos.append({
                "arquivo": filename,
                "analise": analise_individual
            })

        else:
            print(f"  ❌ Não foi possível extrair texto do arquivo: {filename}. Pulando análise deste arquivo.")


    # Exibir todos os resultados acumulados (ou resumir)
    if resultados_analise_arquivos:
        print("\n--- RESUMO DA ANÁLISE DE TODOS OS ARQUIVOS DE RENDIMENTO ENVIADOS ---\n")
        # Concatena os resultados para exibição
        resumo_total = ""
        for resultado in resultados_analise_arquivos:
            resumo_total += f"--- Análise para {resultado['arquivo']} ---\n"
            resumo_total += resultado['analise'] + "\n\n" # Adiciona a análise individual
        display(to_markdown(resumo_total)) # Exibe o resumo
        print("----------------------------------------------------------------------\n\n")
    else:
      # Esta mensagem só aparecerá se o usuário UPOU arquivos, mas NENHUM deles pôde ser lido.
      print("\n❌ Nenhum dos arquivos enviados pôde ser processado ou analisado com sucesso.")
      # --- Informar nome dos arquivos ---
      print("Maravilha! Recebemos seus arquivos.")

      # informando o nome dos arquivos enviados
      print("\nEsses foram os arquivos enviados e que vamos avaliar")

      for filename in upload_rendimentos.keys():
          print(f"- {filename}")
      print("-" * 30) # Linha separadora para clareza

      # Mensagem de espera mais específica
      print("\n-- AGUARDE (Analisando rendimentos dos arquivos enviados) --\n")

      # Lista para armazenar os resultados da análise de cada arquivo
      resultados_analise_arquivos = []

      for filename in upload_rendimentos.keys(): # Itera sobre todos os arquivos enviados
          print(f"Processando arquivo: {filename}...") # Informa qual arquivo está processando
          pdf_text = read_pdf(filename) # Lê o texto deste arquivo específico

          if pdf_text:
              # Chama o agente analisador para CADA arquivo
              print(f"  ✅ Texto extraído. Enviando para análise do agente...")
              analise_individual = agente_analisador_rendimentos(ano, pdf_text)

              # Armazenar o resultado
              resultados_analise_arquivos.append({
                  "arquivo": filename,
                  "analise": analise_individual
              })

          else:
              print(f"  ❌ Não foi possível extrair texto do arquivo: {filename}. Pulando análise deste arquivo.")


      # Exibir todos os resultados acumulados (ou resumir)
      if resultados_analise_arquivos:
          print("\n--- RESUMO DA ANÁLISE DE TODOS OS ARQUIVOS DE RENDIMENTO ENVIADOS ---\n")
          # Concatena os resultados para exibição
          resumo_total = ""
          for resultado in resultados_analise_arquivos:
              resumo_total += f"--- Análise para {resultado['arquivo']} ---\n"
              resumo_total += resultado['analise'] + "\n\n" # Adiciona a análise individual
          display(to_markdown(resumo_total)) # Exibe o resumo
          print("----------------------------------------------------------------------\n\n")
      else:
          # Esta mensagem só aparecerá se o usuário UPOU arquivos, mas NENHUM deles pôde ser lido.
          print("\n❌ Nenhum dos arquivos enviados pôde ser processado ou analisado com sucesso.")

  ######################################################################
  # --- Pedindo ao usuário para fazer upload dos arquivos de notas --- #
  ######################################################################

  print("\n📄 Agora, por favor, faça o upload dos seus arquivos de notas fiscais em formato PDF.")
  print("\n👉 Clique no botão 'Escolher arquivos' que aparecerá abaixo para selecionar os documentos.")
  print("\n✅ Você pode enviar um ou mais arquivos de uma vez.")

  #solicitando o arquivo
  upload_notas = files.upload()

  if not upload_notas:
    print("❌ Nenhum arquivo de nota fiscal foi enviado. Por favor, tente novamente.")
  else:
    # --- Informar nome dos arquivos de notas fiscais ---
    print("Maravilha! Recebemos seus arquivos de notas fiscais.")

    # informando o nome dos arquivos enviados
    print("\nEsses foram os arquivos de notas fiscais enviados e que vamos avaliar")
    for filename in upload_notas.keys(): # <--- Usar upload_notas aqui
        print(f"- {filename}")
    print("-" * 30) # Linha separadora para clareza

    # Mensagem de espera mais específica para notas fiscais
    print("\n-- AGUARDE (Analisando notas fiscais dos arquivos enviados) --\n")

    # Lista para armazenar os resultados da análise de cada arquivo de nota
    resultados_analise_notas = [] # <--- Nova lista para resultados de notas

    for filename in upload_notas.keys(): # <--- Iterar sobre upload_notas
        print(f"Processando arquivo de nota: {filename}...") # Mensagem específica
        pdf_text_nota = read_pdf(filename) # <--- Variável para texto da nota

        if pdf_text_nota: # <--- Usar texto da nota
            # Chama o agente analisador para CADA arquivo de nota
            print(f"  ✅ Texto de nota fiscal extraído. Enviando para análise do agente de notas fiscais...")
            # *** CHAMAR O TERCEIRO AGENTE AQUI ***
            analise_individual_nota = agente_analisador_notas(ano, pdf_text_nota) # <--- Chamar o agente_analisador_notas

            # Armazenar o resultado
            resultados_analise_notas.append({
                "arquivo": filename,
                "analise": analise_individual_nota
            })

        else:
            print(f"  ❌ Não foi possível extrair texto do arquivo de nota fiscal: {filename}. Pulando análise deste arquivo.")


    # Exibir todos os resultados acumulados da análise de notas
    if resultados_analise_notas:
        print("\n--- RESUMO DA ANÁLISE DE TODOS OS ARQUIVOS DE NOTAS FISCAIS ENVIADOS ---\n")
        # Concatena os resultados para exibição
        resumo_total_notas = "" # <--- Nova variável para resumo de notas
        for resultado_nota in resultados_analise_notas: # <--- Iterar sobre resultados_analise_notas
            resumo_total_notas += f"--- Análise para {resultado_nota['arquivo']} ---\n"
            resumo_total_notas += resultado_nota['analise'] + "\n\n" # Adiciona a análise individual
        display(to_markdown(resumo_total_notas)) # <--- Exibir resumo de notas
        print("--------------------------------------------------------------------------\n\n")
    else:
      # Esta mensagem só aparecerá se o usuário UPOU arquivos de nota, mas NENHUM deles pôde ser lido.
      print("\n❌ Nenhum dos arquivos de notas fiscais enviados pôde ser processado ou analisado com sucesso.")


🚀 Iniciando o Assistente de Declaração do Imposto de Renda - DeclaraAI 🚀
❓ Por favor, digite o ano que gostaria de consultar a declaração do imposto de renda: 2025
Maravilha! Vamos buscar informações principais para sua declaração do imposto de renda de: 2025. 



-- AGUARDE (Buscando informações gerais sobre IR para o ano) --


-- Resultado do Agente com Informações de IR --



> Ok, vou buscar as informações mais recentes e relevantes sobre o Imposto de Renda de 2025 para te ajudar com a declaração.
> 
> 
> ## Imposto de Renda 2025: Guia Essencial
> 
> ### Principais Regras
> 
> *   **Prazo Final:** O prazo para entrega da Declaração do Imposto de Renda 2025 (ano-calendário 2024) vai de **17 de março a 30 de maio de 2025**, terminando às 23h59min59s.
> *   **Obrigatoriedade:** Estão obrigados a declarar o IRPF em 2025 os contribuintes que:
>    *   Receberam rendimentos tributáveis superiores a **R$ 33.888,00** em 2024.
>    *   Obtiveram rendimentos isentos, não tributáveis ou tributados exclusivamente na fonte acima de **R$ 200.000,00**.
>    *   Realizaram operações na bolsa de valores acima de **R$ 40.000,00** ou tiveram ganhos líquidos sujeitos à tributação.
>    *   Tiveram receita bruta superior a **R$ 169.440,00** em atividade rural.
>    *   Possuíam, até 31 de dezembro de 2024, bens ou direitos acima de **R$ 800.000,00**.
>    *   Passaram a ser residentes no Brasil em 2024 e permaneceram nessa condição até o final do ano.
>    *   Optaram por declarar bens no exterior.
> *   **Declaração Pré-Preenchida:** A declaração pré-preenchida estará disponível a partir de **1º de abril**. Para acessá-la, é necessário possuir conta gov.br nos níveis ouro ou prata.
> *   **Prioridade na Restituição:**
>    *   A prioridade para recebimento da restituição é para contribuintes que utilizaram a declaração pré-preenchida e optaram por receber a restituição via Pix.
>    *   A ordem de prioridade é:
>        1.  Idosos com mais de 80 anos.
>        2.  Contribuintes entre 60 e 79 anos.
>        3.  Pessoas com deficiência ou doença grave.
>        4.  Professores.
>        5.  Contribuintes que utilizaram a declaração pré-preenchida e optaram por receber a restituição por Pix.
>        6.  Demais contribuintes.
> 
> ### Calendário de Restituição
> 
> Os pagamentos da restituição do IR 2025 serão realizados em 5 lotes:
> 
> *   **1º lote:** 30 de maio
> *   **2º lote:** 30 de junho
> *   **3º lote:** 31 de julho
> *   **4º lote:** 29 de agosto
> *   **5º lote:** 30 de setembro
> 
> ### Novidades para 2025
> 
> *   **Limite de Isenção:** A faixa de isenção foi ajustada. Quem ganha até dois salários mínimos (R$ 3.036,00 mensais) não precisa pagar Imposto de Renda.
> *   **Obrigatoriedade para Residentes no Exterior:** A declaração passa a ser obrigatória para quem possui rendimentos de aplicações financeiras no exterior ou lucros e dividendos de entidades controladas no exterior.
> *   **Códigos de Bens e Direitos:** Houve inclusão de novos códigos para bens e direitos, como garagem avulsa, joias, holding patrimonial e leasing com opção de compra.
> *   **Atualização de Imóveis:** Quem atualizou o valor de mercado de bens imóveis e pagou o imposto definitivo de 4% sobre o ganho de capital também deve declarar.
> 
> ### Deduções Possíveis
> 
> Algumas despesas podem ser deduzidas do Imposto de Renda, reduzindo o valor a ser pago ou aumentando a restituição:
> 
> *   **Gastos com Educação:** Inclui educação infantil, ensino fundamental, ensino médio, educação superior (graduação, pós-graduação, mestrado, doutorado e especialização) e educação profissional (ensino técnico e tecnológico). Existe um limite anual para dedução com instrução.
> *   **Gastos com Saúde:** Despesas médicas podem ser deduzidas integralmente, desde que comprovadas com notas fiscais e recibos.
> *   **Dependentes:** É possível deduzir um valor fixo por dependente (R$ 2.275,08 por dependente).
> *   **Pensão Alimentícia:** Pode ser deduzida integralmente, desde que o valor seja estabelecido por decisão judicial ou acordo homologado judicialmente.
> *   **Doações:** Doações feitas a fundos municipais, estaduais ou programas de incentivo nacional podem ser deduzidas em até 6% do imposto devido.
> *   **Previdência Privada:** Contribuições para PGBL (Plano Gerador de Benefício Livre) podem ser deduzidas até o limite de 12% da renda bruta anual tributável.
> 
> ### Recomendações Importantes
> 
> *   **Atenção aos Números:** Evite erros de digitação ao preencher a declaração.
> *   **Dependentes:** Organize a declaração de dependentes entre os membros da família para evitar duplicidade.
> *   **Informe Todas as Receitas:** Não omita nenhum rendimento, incluindo os de dependentes.
> *   **Gastos Dedutíveis:** Inclua apenas os gastos dedutíveis permitidos, dentro dos limites estabelecidos.
> *   **Documentação:** Guarde todos os comprovantes de rendimentos e despesas dedutíveis por, no mínimo, cinco anos.
> 
> Lembre-se de que o programa para preencher a declaração já está disponível para download.
> 


---------------------------------------------



📄 Agora, por favor, faça o upload dos seus arquivos de informe de rendimentos em formato PDF.

👉 Clique no botão 'Escolher arquivos' que aparecerá abaixo para selecionar os documentos.

✅ Você pode enviar um ou mais arquivos de uma vez.


Saving Xp-rendimentos.pdf to Xp-rendimentos (3).pdf
Maravilha! Recebemos seus arquivos.

Esses foram os arquivos enviados e que vamos avaliar
- Xp-rendimentos (3).pdf
------------------------------

-- AGUARDE (Analisando rendimentos dos arquivos enviados) --

Processando arquivo: Xp-rendimentos (3).pdf...
  ✅ Texto extraído. Enviando para análise do agente...

--- RESUMO DA ANÁLISE DE TODOS OS ARQUIVOS DE RENDIMENTO ENVIADOS ---



> --- Análise para Xp-rendimentos (3).pdf ---
> Certo, analisei o texto do relatório auxiliar de proventos pagos de 2024. É importante notar que este é um *relatório auxiliar para simples conferência* e, conforme indicado no próprio documento, você deve utilizar os *informes de rendimentos oficiais* enviados pelas fontes pagadoras (as empresas/fundos) para fazer sua declaração de Imposto de Renda 2025 (referente ao ano-calendário de 2024).
> 
> A XP Investimentos CCTVM S/A (CNPJ: 02.332.886/0001-04) é a custodiante que gerou este relatório. As fontes pagadoras dos proventos listados são as próprias empresas (B3, INTERCO, TAESA, VIVARA S.A.) e os Fundos de Investimento Imobiliário (FIIs).
> 
> Abaixo estão as informações extraídas, agregadas por fonte pagadora e tipo de rendimento, com indicação de onde declarar no programa da Receita Federal:
> 
> *   **Fonte Pagadora:** B3
>     *   **Tipo de Rendimento:** Dividendos (isentos)
>     *   **Valor Bruto:** R$ 17,78
>     *   **Valor IR Retido:** R$ 0,00
>     *   **Onde Declarar:** Ficha "Rendimentos Isentos e Não Tributáveis", Código 09 - Lucros e dividendos recebidos. (Pode ser necessário o CNPJ da B3 S.A. - Brasil, Bolsa, Balcão).
> 
> *   **Fonte Pagadora:** B3
>     *   **Tipo de Rendimento:** Juros sobre Capital Próprio (JCP)
>     *   **Valor Bruto:** R$ 22,35
>     *   **Valor IR Retido:** R$ 3,33
>     *   **Onde Declarar:** Ficha "Rendimentos Sujeitos à Tributação Exclusiva/Definitiva", Código 10 - Juros sobre Capital Próprio. (Pode ser necessário o CNPJ da B3 S.A. - Brasil, Bolsa, Balcão).
> 
> *   **Fonte Pagadora:** CARTESIA FII
>     *   **Tipo de Rendimento:** Rendimento de FII (geralmente isento, verificar no informe oficial)
>     *   **Valor Bruto:** R$ 361,35
>     *   **Valor IR Retido:** R$ 0,00
>     *   **Onde Declarar:** Ficha "Rendimentos Isentos e Não Tributáveis", Código 26 - Outros rendimentos isentos ou não tributáveis. Especificar "Rendimentos de FII [Nome do FII] (Lei 11.033/04)". (Será necessário o CNPJ do fundo CARTESIA FII, que deve constar no informe oficial do administrador do fundo).
> 
> *   **Fonte Pagadora:** FIINFRARIF
>     *   **Tipo de Rendimento:** Rendimento de FII (geralmente isento, verificar no informe oficial)
>     *   **Valor Bruto:** R$ 41,59
>     *   **Valor IR Retido:** R$ 0,00
>     *   **Onde Declarar:** Ficha "Rendimentos Isentos e Não Tributáveis", Código 26 - Outros rendimentos isentos ou não tributáveis. Especificar "Rendimentos de FII [Nome do FII] (Lei 11.033/04)". (Será necessário o CNPJ do fundo FIINFRARIF, que deve constar no informe oficial do administrador do fundo).
> 
> *   **Fonte Pagadora:** FIICAPISEC
>     *   **Tipo de Rendimento:** Rendimento de FII (geralmente isento, verificar no informe oficial)
>     *   **Valor Bruto:** R$ 89,60
>     *   **Valor IR Retido:** R$ 0,00
>     *   **Onde Declarar:** Ficha "Rendimentos Isentos e Não Tributáveis", Código 26 - Outros rendimentos isentos ou não tributáveis. Especificar "Rendimentos de FII [Nome do FII] (Lei 11.033/04)". (Será necessário o CNPJ do fundo FIICAPISEC, que deve constar no informe oficial do administrador do fundo).
> 
> *   **Fonte Pagadora:** FIIHECTARE
>     *   **Tipo de Rendimento:** Rendimento de FII (geralmente isento, verificar no informe oficial)
>     *   **Valor Bruto:** R$ 20,98
>     *   **Valor IR Retido:** R$ 0,00
>     *   **Onde Declarar:** Ficha "Rendimentos Isentos e Não Tributáveis", Código 26 - Outros rendimentos isentos ou não tributáveis. Especificar "Rendimentos de FII [Nome do FII] (Lei 11.033/04)". (Será necessário o CNPJ do fundo FIIHECTARE, que deve constar no informe oficial do administrador do fundo).
> 
> *   **Fonte Pagadora:** FIIHTOPFOF3
>     *   **Tipo de Rendimento:** Rendimento de FII (geralmente isento, verificar no informe oficial)
>     *   **Valor Bruto:** R$ 41,58
>     *   **Valor IR Retido:** R$ 0,00
>     *   **Onde Declarar:** Ficha "Rendimentos Isentos e Não Tributáveis", Código 26 - Outros rendimentos isentos ou não tributáveis. Especificar "Rendimentos de FII [Nome do FII] (Lei 11.033/04)". (Será necessário o CNPJ do fundo FIIHTOPFOF3, que deve constar no informe oficial do administrador do fundo).
> 
> *   **Fonte Pagadora:** FIIKINEA
>     *   **Tipo de Rendimento:** Rendimento de FII (geralmente isento, verificar no informe oficial)
>     *   **Valor Bruto:** R$ 48,00
>     *   **Valor IR Retido:** R$ 0,00
>     *   **Onde Declarar:** Ficha "Rendimentos Isentos e Não Tributáveis", Código 26 - Outros rendimentos isentos ou não tributáveis. Especificar "Rendimentos de FII [Nome do FII] (Lei 11.033/04)". (Será necessário o CNPJ do fundo FIIKINEA, que deve constar no informe oficial do administrador do fundo).
> 
> *   **Fonte Pagadora:** FIIKIVO
>     *   **Tipo de Rendimento:** Rendimento de FII (geralmente isento, verificar no informe oficial)
>     *   **Valor Bruto:** R$ 601,00
>     *   **Valor IR Retido:** R$ 0,00
>     *   **Onde Declarar:** Ficha "Rendimentos Isentos e Não Tributáveis", Código 26 - Outros rendimentos isentos ou não tributáveis. Especificar "Rendimentos de FII [Nome do FII] (Lei 11.033/04)". (Será necessário o CNPJ do fundo FIIKIVO, que deve constar no informe oficial do administrador do fundo).
> 
> *   **Fonte Pagadora:** FIIMAXIREN
>     *   **Tipo de Rendimento:** Rendimento de FII (geralmente isento, verificar no informe oficial)
>     *   **Valor Bruto:** R$ 99,00
>     *   **Valor IR Retido:** R$ 0,00
>     *   **Onde Declarar:** Ficha "Rendimentos Isentos e Não Tributáveis", Código 26 - Outros rendimentos isentos ou não tributáveis. Especificar "Rendimentos de FII [Nome do FII] (Lei 11.033/04)". (Será necessário o CNPJ do fundo FIIMAXIREN, que deve constar no informe oficial do administrador do fundo).
> 
> *   **Fonte Pagadora:** FIIPMIS
>     *   **Tipo de Rendimento:** Rendimento de FII (geralmente isento, verificar no informe oficial)
>     *   **Valor Bruto:** R$ 174,66
>     *   **Valor IR Retido:** R$ 0,00
>     *   **Onde Declarar:** Ficha "Rendimentos Isentos e Não Tributáveis", Código 26 - Outros rendimentos isentos ou não tributáveis. Especificar "Rendimentos de FII [Nome do FII] (Lei 11.033/04)". (Será necessário o CNPJ do fundo FIIPMIS, que deve constar no informe oficial do administrador do fundo).
> 
> *   **Fonte Pagadora:** FIIRECRECE
>     *   **Tipo de Rendimento:** Rendimento de FII (geralmente isento, verificar no informe oficial)
>     *   **Valor Bruto:** R$ 54,39
>     *   **Valor IR Retido:** R$ 0,00
>     *   **Onde Declarar:** Ficha "Rendimentos Isentos e Não Tributáveis", Código 26 - Outros rendimentos isentos ou não tributáveis. Especificar "Rendimentos de FII [Nome do FII] (Lei 11.033/04)". (Será necessário o CNPJ do fundo FIIRECRECE, que deve constar no informe oficial do administrador do fundo).
> 
> *   **Fonte Pagadora:** FIIVALORAIP
>     *   **Tipo de Rendimento:** Rendimento de FII (geralmente isento, verificar no informe oficial)
>     *   **Valor Bruto:** R$ 506,50
>     *   **Valor IR Retido:** R$ 0,00
>     *   **Onde Declarar:** Ficha "Rendimentos Isentos e Não Tributáveis", Código 26 - Outros rendimentos isentos ou não tributáveis. Especificar "Rendimentos de FII [Nome do FII] (Lei 11.033/04)". (Será necessário o CNPJ do fundo FIIVALORAIP, que deve constar no informe oficial do administrador do fundo).
> 
> *   **Fonte Pagadora:** FIIVALREIII
>     *   **Tipo de Rendimento:** Rendimento de FII (geralmente isento, verificar no informe oficial)
>     *   **Valor Bruto:** R$ 887,70
>     *   **Valor IR Retido:** R$ 0,00
>     *   **Onde Declarar:** Ficha "Rendimentos Isentos e Não Tributáveis", Código 26 - Outros rendimentos isentos ou não tributáveis. Especificar "Rendimentos de FII [Nome do FII] (Lei 11.033/04)". (Será necessário o CNPJ do fundo FIIVALREIII, que deve constar no informe oficial do administrador do fundo).
> 
> *   **Fonte Pagadora:** FIIVECTIS
>     *   **Tipo de Rendimento:** Rendimento de FII (geralmente isento, verificar no informe oficial)
>     *   **Valor Bruto:** R$ 1.210,00
>     *   **Valor IR Retido:** R$ 0,00
>     *   **Onde Declarar:** Ficha "Rendimentos Isentos e Não Tributáveis", Código 26 - Outros rendimentos isentos ou não tributáveis. Especificar "Rendimentos de FII [Nome do FII] (Lei 11.033/04)". (Será necessário o CNPJ do fundo FIIVECTIS, que deve constar no informe oficial do administrador do fundo).
> 
> *   **Fonte Pagadora:** FIIXPLOG
>     *   **Tipo de Rendimento:** Rendimento de FII (geralmente isento, verificar no informe oficial)
>     *   **Valor Bruto:** R$ 93,60
>     *   **Valor IR Retido:** R$ 0,00
>     *   **Onde Declarar:** Ficha "Rendimentos Isentos e Não Tributáveis", Código 26 - Outros rendimentos isentos ou não tributáveis. Especificar "Rendimentos de FII [Nome do FII] (Lei 11.033/04)". (Será necessário o CNPJ do fundo FIIXPLOG, que deve constar no informe oficial do administrador do fundo).
> 
> *   **Fonte Pagadora:** FIIXPMALLS
>     *   **Tipo de Rendimento:** Rendimento de FII (geralmente isento, verificar no informe oficial)
>     *   **Valor Bruto:** R$ 27,30
>     *   **Valor IR Retido:** R$ 0,00
>     *   **Onde Declarar:** Ficha "Rendimentos Isentos e Não Tributáveis", Código 26 - Outros rendimentos isentos ou não tributáveis. Especificar "Rendimentos de FII [Nome do FII] (Lei 11.033/04)". (Será necessário o CNPJ do fundo FIIXPMALLS, que deve constar no informe oficial do administrador do fundo).
> 
> *   **Fonte Pagadora:** INTERCO
>     *   **Tipo de Rendimento:** Dividendos (isentos)
>     *   **Valor Bruto:** R$ 8,28
>     *   **Valor IR Retido:** R$ 0,00
>     *   **Onde Declarar:** Ficha "Rendimentos Isentos e Não Tributáveis", Código 09 - Lucros e dividendos recebidos. (Será necessário o CNPJ da empresa INTERCO, que deve constar no informe oficial).
> 
> *   **Fonte Pagadora:** TAESA
>     *   **Tipo de Rendimento:** Dividendos (isentos)
>     *   **Valor Bruto:** R$ 111,27
>     *   **Valor IR Retido:** R$ 0,00
>     *   **Onde Declarar:** Ficha "Rendimentos Isentos e Não Tributáveis", Código 09 - Lucros e dividendos recebidos. (Será necessário o CNPJ da empresa TAESA, que deve constar no informe oficial).
> 
> *   **Fonte Pagadora:** TAESA
>     *   **Tipo de Rendimento:** Juros sobre Capital Próprio (JCP)
>     *   **Valor Bruto:** R$ 40,47
>     *   **Valor IR Retido:** R$ 6,06
>     *   **Onde Declarar:** Ficha "Rendimentos Sujeitos à Tributação Exclusiva/Definitiva", Código 10 - Juros sobre Capital Próprio. (Será necessário o CNPJ da empresa TAESA, que deve constar no informe oficial).
> 
> *   **Fonte Pagadora:** VIVARA S.A.
>     *   **Tipo de Rendimento:** Dividendos (isentos)
>     *   **Valor Bruto:** R$ 0,37
>     *   **Valor IR Retido:** R$ 0,00
>     *   **Onde Declarar:** Ficha "Rendimentos Isentos e Não Tributáveis", Código 09 - Lucros e dividendos recebidos. (Será necessário o CNPJ da empresa VIVARA S.A., que deve constar no informe oficial).
> 
> *   **Eventos que NÃO são Rendimentos Tributáveis/Isentos:**
>     *   **Tipo de Evento:** Amortização de FII (FIINFRARIF: R$ 38,40)
>     *   **Observação:** Conforme o relatório, amortizações não são proventos pagos e servem para ajustar o custo médio do ativo.
>     *   **Onde Lançar (Relevância para IR):** Não são lançadas nas fichas de Rendimentos. São relevantes para a Ficha "Bens e Direitos" (para ajustar o custo de aquisição do FII) e para o cálculo de Ganho de Capital caso haja venda futura das cotas. Mantenha este valor registrado para controle de custo médio.
> 
> **Outras informações relevantes do relatório:**
> 
> *   **Ano Calendário:** 2024
> *   **Ano de Referência para Declaração:** 2025
> *   **Declarante:** MARIANA DECASSIA CAVELANHA ALVES (CPF: 310.492.488-05)
> *   **Custodiante (intermediário):** XP INVESTIMENTOS CCTVM S/A (CNPJ: 02.332.886/0001-04)
> *   **Conta na XP:** AGÊNCIA: 0001 CONTA: 92931
> *   **Importante:** Este relatório *não* inclui valores recebidos durante custódia remunerada (aluguel de ações/BTC). Esses valores devem ser obtidos no site do CEI (B3).
> *   **Recomendação do relatório:** Obter os informes de rendimentos oficiais diretamente das fontes pagadoras (administradores dos fundos e empresas).
> 
> Lembre-se de que os CNPJs das fontes pagadoras (empresas e fundos) são essenciais para o preenchimento correto da declaração e devem ser obtidos nos informes oficiais.
> 
> 


----------------------------------------------------------------------



📄 Agora, por favor, faça o upload dos seus arquivos de notas fiscais em formato PDF.

👉 Clique no botão 'Escolher arquivos' que aparecerá abaixo para selecionar os documentos.

✅ Você pode enviar um ou mais arquivos de uma vez.


Saving 10-nfs_ver990.2421794364750055.pdf to 10-nfs_ver990.2421794364750055 (2).pdf
Saving 09-nfs_ver990.6632607643285036.pdf to 09-nfs_ver990.6632607643285036 (2).pdf
Maravilha! Recebemos seus arquivos de notas fiscais.

Esses foram os arquivos de notas fiscais enviados e que vamos avaliar
- 10-nfs_ver990.2421794364750055 (2).pdf
- 09-nfs_ver990.6632607643285036 (2).pdf
------------------------------

-- AGUARDE (Analisando notas fiscais dos arquivos enviados) --

Processando arquivo de nota: 10-nfs_ver990.2421794364750055 (2).pdf...
  ✅ Texto de nota fiscal extraído. Enviando para análise do agente de notas fiscais...
Processando arquivo de nota: 09-nfs_ver990.6632607643285036 (2).pdf...
  ✅ Texto de nota fiscal extraído. Enviando para análise do agente de notas fiscais...

--- RESUMO DA ANÁLISE DE TODOS OS ARQUIVOS DE NOTAS FISCAIS ENVIADOS ---



> --- Análise para 10-nfs_ver990.2421794364750055 (2).pdf ---
> Com base na análise do texto fornecido, seguem as informações relevantes para a sua declaração de Imposto de Renda 2025 (ano-base 2024):
> 
> Resumo da Análise:
> Este documento é uma Nota Fiscal Eletrônica de Serviços (NFS-e) relacionada a serviços de educação infantil prestados em 2024. Despesas com educação são, em geral, dedutíveis na declaração de Imposto de Renda, respeitando os limites estabelecidos pela legislação.
> 
> Informações Extraídas:
> 
> Tipo de Nota Fiscal:
> Serviço de Educação (Ensino regular pré-escolar, fundamental, médio e superior).
> 
> Informações do Prestador do Serviço:
> Nome/Razão Social: SOCIEDADE EDUCACIONAL TRAVESSIA LTDA
> CNPJ: 32.371.225/0001-40
> Endereço: AV DAS HORTÊNCIAS ,524 - SEIXAS - JD. DOS FUNDOS:HC 1, CEP: 15061-080, SAO JOSE DO RIO PRETO - SP.
> Outras informações relevantes: Inscrição Municipal: 3574510, Telefone: (17)9157-8372, e-mail: patricia.maia87@gmail.com.
> 
> Informações do Tomador do Serviço (Quem pagou/será declarado):
> Nome/Razão Social: Mariana de Cássia Cavelanha Alves
> CPF: 592.808.238-09
> Endereço: JOSÉ DA SILVA SÉ ,1000 - VILLAGE DAMHA RIO PRETO III, CEP: 15056-110, SAO JOSE DO RIO PRETO - SP.
> Outras informações relevantes: Telefone: 17982008658, e-mail: marianaa@gmail.com.
> 
> Verificação da Data de Emissão:
> Data de Emissão: 07/10/2024. Esta data está dentro do ano de referência (2024) para a declaração que será entregue em 2025. Portanto, a despesa é elegível para ser declarada.
> 
> Valores Pagos:
> Valor Total da Nota Fiscal: R$ 1.068,00.
> Valor dos Serviços: R$ 1.068,00.
> 
> Impostos Retidos na Fonte:
> Nenhum imposto retido na fonte foi informado na nota (IR R$ 0,00, INSS R$ 0,00, CSLL R$ 0,00, PIS, COFINS).
> 
> Descrição do Serviço:
> Ensino regular pré-escolar, fundamental, médio e superior. Especificamente "Educação Infantil" para o aluno Theo da Silva Cavelanha Alves.
> 
> Outras Informações Relevantes:
> A nota identifica o aluno beneficiado pela educação: Theo da Silva Cavelanha Alves, CPF 310.492.488-05. Este dado é importante se o aluno for seu dependente ou alimentando.
> Número da NFS-e: 3235.
> Competência: 7/9/2024 (Mês de referência do serviço).
> 
> Onde Declarar no IR 2025 (Ano-base 2024):
> Esta despesa com educação é provavelmente dedutível. Você deverá declará-la na ficha "Pagamentos Efetuados".
> 
> Campos que você precisará preencher na declaração (ficha "Pagamentos Efetuados"):
> 
> *   Código: Escolha o código correspondente a "Educação - Ensino Infantil, Fundamental, Médio e Superior" (verifique a lista exata de códigos na sua versão do programa/aplicativo da Receita Federal).
> *   Descrição: Informe detalhes como "Mensalidade - Educação Infantil para [Nome Completo do Aluno]", mencionando o nome do aluno beneficiado e o período/serviço.
> *   Nome do Prestador do Serviço: SOCIEDADE EDUCACIONAL TRAVESSIA LTDA.
> *   CNPJ do Prestador do Serviço: 32.371.225/0001-40.
> *   Valor Pago: 1.068,00.
> *   Valor Dedução/Reembolso (se houver): 0,00 (neste caso, conforme a nota).
> *   Se o aluno Theo da Silva Cavelanha Alves for seu dependente ou alimentando, você precisará informar o nome completo e o CPF dele no campo específico dentro do lançamento deste pagamento na ficha "Pagamentos Efetuados". Certifique-se também de que ele esteja devidamente listado na ficha "Dependentes" ou "Alimentandos", conforme o caso.
> 
> 
> --- Análise para 09-nfs_ver990.6632607643285036 (2).pdf ---
> Okay, vamos analisar o texto desta Nota Fiscal Eletrônica de Serviços (NFS-e) para a sua declaração de Imposto de Renda do ano de referência 2025 (que se refere aos rendimentos e despesas de 2024).
> 
> **Análise da Nota Fiscal:**
> 
> *   **Tipo de Documento:** Nota Fiscal Eletrônica de Serviços (NFS-e).
> *   **Ano de Referência:** A nota foi emitida em 06/09/2024 e a competência é 6/8/2024. Como a declaração de 2025 se refere ao ano-calendário de 2024, esta despesa é **válida** para ser declarada.
> *   **Tipo de Serviço:** Pelo código e descrição "8.01 / 08.01.00 - Ensino regular pré-escolar, fundamental, médio e superior." e "Educação Infantil", trata-se de um **serviço de educação**.
> 
> **Informações Extraídas para a Declaração:**
> 
> *   **Prestador do Serviço:**
>     *   Nome/Razão Social: SOCIEDADE EDUCACIONAL TRAVESSIA LTDA
>     *   CNPJ: 32.371.225/0001-40
>     *   Endereço: AV DAS HORTÊNCIAS ,524 - SEIXAS - JD. DOS FUNDOS:HC 1, CEP: 15061-080, SAO JOSE DO RIO PRETO - SP
> *   **Tomador do Serviço (Pagador):**
>     *   Nome: Mariana de Cássia Cavelanha Alves
>     *   CPF: 592.808.238-09 (Este é o CPF do declarante ou de um de seus dependentes/alimentandos que realizou o pagamento)
> *   **Beneficiário do Serviço (Aluno):**
>     *   Nome: Theo da Silva Cavelanha Alves
>     *   CPF: 31049248805
> *   **Data de Emissão:** 06/09/2024
> *   **Descrição do Serviço:** Educação Infantil (referente à competência de Agosto/2024, pelo valor de R$ 1068,00)
> *   **Valores Pagos:** R$ 1.068,00
> *   **Impostos Retidos na Fonte:** A nota indica R$ 0,00 para IR, INSS, CSLL, ISSQN Retido e Retenções Federais. Portanto, não há impostos retidos a serem informados na declaração com base nesta nota.
> *   **Outras Informações Relevantes:** O nome e CPF do aluno são essenciais, pois a dedução é feita por beneficiário da despesa educacional, que precisa ser seu dependente ou alimentando na declaração.
> 
> **Resumo para a Declaração de Imposto de Renda:**
> 
> Esta nota fiscal refere-se a uma despesa com **Educação Infantil** paga em 2024. Despesas com educação são **dedutíveis** na declaração de ajuste anual, sujeitas a um limite anual por indivíduo (declarado como titular, dependente ou alimentando).
> 
> Você deverá informar esta despesa na seção **"Pagamentos Efetuados"** da sua declaração.
> 
> Os campos que você precisará preencher são:
> 
> *   **Código:** 01 - Instrução - Educação Infantil, Fundamental, Médio e Superior
> *   **Descrição:** Informe o nome da instituição de ensino, o nome e CPF do aluno (Theo da Silva Cavelanha Alves, CPF 31049248805) e a qual mês/período e serviço a nota se refere (Ex: Educação Infantil - Mensalidade Agosto/2024).
> *   **Nome do Prestador:** SOCIEDADE EDUCACIONAL TRAVESSIA LTDA
> *   **CNPJ do Prestador:** 32.371.225/0001-40
> *   **Valor Pago:** 1.068,00
> *   **Parcela Não Dedutível / Valor Reembolsado:** Deixe em branco ou 0,00, a menos que alguma parte tenha sido reembolsada ou não seja dedutível por outro motivo.
> *   Marque se o pagamento se refere ao Titular ou a um Dependente/Alimentando. Lembre-se que Theo da Silva Cavelanha Alves (o aluno) precisa estar declarado como seu dependente ou alimentando para que esta despesa seja dedutível.
> 
> Lembre-se que o valor total das despesas com educação dedutíveis é limitado por lei para cada indivíduo (titular, dependente ou alimentando). O programa da Receita Federal calculará automaticamente este limite.
> 
> 


--------------------------------------------------------------------------


